In [55]:
# read the parquet file containing all (normalized) metrics
import polars.selectors as cs
import polars as pl
import plotly as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, HTML, clear_output
from dash import Dash, dcc, html, Input, Output, callback, jupyter_dash, State, no_update
from dash.exceptions import PreventUpdate
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import io
from PyPDF2 import PdfMerger
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split 


In [2]:
parquet_path = "./parquet4visual.parquet"

In [3]:
df = pl.read_parquet(parquet_path)

In [4]:
pl.Config.set_tbl_cols(250)
df.head

<bound method DataFrame.head of shape: (449_537, 232)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───

In [25]:
pca_df_list = []

In [ ]:
#keep only relevant columns for pca
for coln in df.columns:
    if coln in  ['PSV-99', 'position_grouped', 'competition_region_division'] or 'P90' in coln:
        pca_df_list.append(df.select(coln))
    else:
        continue

In [28]:
pca_df = pl.concat(pca_df_list, how="horizontal")

In [29]:
pca_df.head()

P90 Count HSR,P90 Count HSR 1,P90 Count HSR 2,P90 Count HSR OTIP,P90 Count HSR OTIP 1,P90 Count HSR OTIP 2,P90 Count HSR TIP,P90 Count HSR TIP 1,P90 Count HSR TIP 2,P90 Count High Acceleration,P90 Count High Acceleration 1,P90 Count High Acceleration 2,P90 Count High Acceleration OTIP,P90 Count High Acceleration OTIP 1,P90 Count High Acceleration OTIP 2,P90 Count High Acceleration TIP,P90 Count High Acceleration TIP 1,P90 Count High Acceleration TIP 2,P90 Count High Deceleration,P90 Count High Deceleration 1,P90 Count High Deceleration 2,P90 Count High Deceleration OTIP,P90 Count High Deceleration OTIP 1,P90 Count High Deceleration OTIP 2,P90 Count High Deceleration TIP,P90 Count High Deceleration TIP 1,P90 Count High Deceleration TIP 2,P90 Count Medium Acceleration,P90 Count Medium Acceleration 1,P90 Count Medium Acceleration 2,P90 Count Medium Acceleration OTIP,P90 Count Medium Acceleration OTIP 1,P90 Count Medium Acceleration OTIP 2,P90 Count Medium Acceleration TIP,P90 Count Medium Acceleration TIP 1,P90 Count Medium Acceleration TIP 2,P90 Count Medium Deceleration,P90 Count Medium Deceleration 1,P90 Count Medium Deceleration 2,P90 Count Medium Deceleration OTIP,P90 Count Medium Deceleration OTIP 1,P90 Count Medium Deceleration OTIP 2,P90 Count Medium Deceleration TIP,P90 Count Medium Deceleration TIP 1,P90 Count Medium Deceleration TIP 2,P90 Count Sprint,P90 Count Sprint 1,P90 Count Sprint 2,P90 Count Sprint OTIP,P90 Count Sprint OTIP 1,P90 Count Sprint OTIP 2,P90 Count Sprint TIP,P90 Count Sprint TIP 1,P90 Count Sprint TIP 2,P90 Distance,P90 Distance 1,P90 Distance 2,P90 Distance OTIP,P90 Distance OTIP 1,P90 Distance OTIP 2,P90 Distance TIP,P90 Distance TIP 1,P90 Distance TIP 2,P90 HSR Distance,P90 HSR Distance 1,P90 HSR Distance 2,P90 HSR Distance OTIP,P90 HSR Distance OTIP 1,P90 HSR Distance OTIP 2,P90 HSR Distance TIP,P90 HSR Distance TIP 1,P90 HSR Distance TIP 2,P90 Minutes,P90 Minutes 1,P90 Minutes 2,P90 Minutes OTIP,P90 Minutes OTIP 1,P90 Minutes OTIP 2,P90 Minutes TIP,P90 Minutes TIP 1,P90 Minutes TIP 2,P90 Running Distance,P90 Running Distance 1,P90 Running Distance 2,P90 Running Distance OTIP,P90 Running Distance OTIP 1,P90 Running Distance OTIP 2,P90 Running Distance TIP,P90 Running Distance TIP 1,P90 Running Distance TIP 2,P90 Sprinting Distance,P90 Sprinting Distance 1,P90 Sprinting Distance 2,P90 Sprinting Distance OTIP,P90 Sprinting Distance OTIP 1,P90 Sprinting Distance OTIP 2,P90 Sprinting Distance TIP,P90 Sprinting Distance TIP 1,P90 Sprinting Distance TIP 2,P90 competition.id,P90 season.end_year,P90 season.id,P90 season.start_year,PSV-99,competition_region_division,position_grouped
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
54.951878,62.990456,47.291835,86.027551,83.743842,88.439306,70.63197,89.108911,48.736462,3.725551,5.726405,1.818917,0.0,0.0,0.0,0.0,0.0,0.0,10.245265,15.270414,5.45675,10.120888,14.778325,5.202312,11.152416,13.709063,8.122744,93.138777,99.257688,87.308003,118.920439,118.226601,119.653179,81.784387,82.254379,81.227437,82.893511,80.169671,85.489086,86.027551,83.743842,88.439306,63.197026,54.836253,73.104693,4.656939,5.726405,3.637833,5.060444,4.926108,5.202312,7.434944,6.854532,8.122744,12132.257063,12998.939555,11306.386419,14490.581951,14694.581281,14275.144509,12758.364312,12845.392232,12655.234657,693.883887,738.706257,651.172191,1120.888389,1108.374384,1134.104046,966.542751,952.779893,982.851986,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,2249.301459,2584.517497,1929.870655,3805.454034,4147.783251,3443.930636,2457.249071,2529.322163,2371.841155,137.84539,175.609756,101.859337,113.859994,133.004926,93.6416

In [ ]:
# manually remove leftover irelevant columns
pca_df.drop([
    'P90 Minutes',
    'P90 Minutes 1',
    'P90 Minutes 2',
    'P90 Minutes OTIP',
    'P90 Minutes OTIP 1',
    'P90 Minutes OTIP 2',
    'P90 Minutes TIP',
    'P90 Minutes TIP 1',
    'P90 Minutes TIP 2',
    'P90 competition.id',
    'P90 season.end_year',
    'P90 season.id',
    'P90 season.start_year'
])

P90 Count HSR,P90 Count HSR 1,P90 Count HSR 2,P90 Count HSR OTIP,P90 Count HSR OTIP 1,P90 Count HSR OTIP 2,P90 Count HSR TIP,P90 Count HSR TIP 1,P90 Count HSR TIP 2,P90 Count High Acceleration,P90 Count High Acceleration 1,P90 Count High Acceleration 2,P90 Count High Acceleration OTIP,P90 Count High Acceleration OTIP 1,P90 Count High Acceleration OTIP 2,P90 Count High Acceleration TIP,P90 Count High Acceleration TIP 1,P90 Count High Acceleration TIP 2,P90 Count High Deceleration,P90 Count High Deceleration 1,P90 Count High Deceleration 2,P90 Count High Deceleration OTIP,P90 Count High Deceleration OTIP 1,P90 Count High Deceleration OTIP 2,P90 Count High Deceleration TIP,P90 Count High Deceleration TIP 1,P90 Count High Deceleration TIP 2,P90 Count Medium Acceleration,P90 Count Medium Acceleration 1,P90 Count Medium Acceleration 2,P90 Count Medium Acceleration OTIP,P90 Count Medium Acceleration OTIP 1,P90 Count Medium Acceleration OTIP 2,P90 Count Medium Acceleration TIP,P90 Count Medium Acceleration TIP 1,P90 Count Medium Acceleration TIP 2,P90 Count Medium Deceleration,P90 Count Medium Deceleration 1,P90 Count Medium Deceleration 2,P90 Count Medium Deceleration OTIP,P90 Count Medium Deceleration OTIP 1,P90 Count Medium Deceleration OTIP 2,P90 Count Medium Deceleration TIP,P90 Count Medium Deceleration TIP 1,P90 Count Medium Deceleration TIP 2,P90 Count Sprint,P90 Count Sprint 1,P90 Count Sprint 2,P90 Count Sprint OTIP,P90 Count Sprint OTIP 1,P90 Count Sprint OTIP 2,P90 Count Sprint TIP,P90 Count Sprint TIP 1,P90 Count Sprint TIP 2,P90 Distance,P90 Distance 1,P90 Distance 2,P90 Distance OTIP,P90 Distance OTIP 1,P90 Distance OTIP 2,P90 Distance TIP,P90 Distance TIP 1,P90 Distance TIP 2,P90 HSR Distance,P90 HSR Distance 1,P90 HSR Distance 2,P90 HSR Distance OTIP,P90 HSR Distance OTIP 1,P90 HSR Distance OTIP 2,P90 HSR Distance TIP,P90 HSR Distance TIP 1,P90 HSR Distance TIP 2,P90 Running Distance,P90 Running Distance 1,P90 Running Distance 2,P90 Running Distance OTIP,P90 Running Distance OTIP 1,P90 Running Distance OTIP 2,P90 Running Distance TIP,P90 Running Distance TIP 1,P90 Running Distance TIP 2,P90 Sprinting Distance,P90 Sprinting Distance 1,P90 Sprinting Distance 2,P90 Sprinting Distance OTIP,P90 Sprinting Distance OTIP 1,P90 Sprinting Distance OTIP 2,P90 Sprinting Distance TIP,P90 Sprinting Distance TIP 1,P90 Sprinting Distance TIP 2,PSV-99,competition_region_division,position_grouped
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
54.951878,62.990456,47.291835,86.027551,83.743842,88.439306,70.63197,89.108911,48.736462,3.725551,5.726405,1.818917,0.0,0.0,0.0,0.0,0.0,0.0,10.245265,15.270414,5.45675,10.120888,14.778325,5.202312,11.152416,13.709063,8.122744,93.138777,99.257688,87.308003,118.920439,118.226601,119.653179,81.784387,82.254379,81.227437,82.893511,80.169671,85.489086,86.027551,83.743842,88.439306,63.197026,54.836253,73.104693,4.656939,5.726405,3.637833,5.060444,4.926108,5.202312,7.434944,6.854532,8.122744,12132.257063,12998.939555,11306.386419,14490.581951,14694.581281,14275.144509,12758.364312,12845.392232,12655.234657,693.883887,738.706257,651.172191,1120.888389,1108.374384,1134.104046,966.542751,952.779893,982.851986,2249.301459,2584.517497,1929.870655,3805.454034,4147.783251,3443.930636,2457.249071,2529.322163,2371.841155,137.84539,175.609756,101.859337,113.859994,133.004926,93.641618,111.524164,75.399848,154.33213,25.7,"""Belgium 1""","""AM"""
36.49028,39.028621,34.152187,51.706658,61.983471,41.80491,70.532915,61.363636,80.357143,4.678241,5.854293,3.594967,3.041568,0.0,5.97213,7.053292,6.818182,7.305195,7.485186,9.757155,5.392451,3.041568,6.198347,0.0,7.053292,6.818182,7.305195,72.044911,70.251

In [45]:
pca_df['position_grouped'].unique()

position_grouped
str
"""RCB|LCB"""
"""RM|LM"""
"""AM"""
"""RW|LW"""
"""CF"""
"""RF|LF"""
"""DM"""
"""CB"""
"""RWB|LWB"""


In [46]:
position_dfs = {}

In [ ]:
# store a dataframe per position in a seperate dataframe, all dataframes stored together in a dictionary
# needed because positions have to be seperated for fair pca analysis
for position in pca_df['position_grouped'].unique():
    position_dfs[position] = pca_df.filter(
        pl.col('position_grouped') == position
    )

In [48]:
position_dfs.keys()

dict_keys(['CB', 'AM', 'CF', 'RF|LF', 'RM|LM', 'RWB|LWB', 'DM', 'RW|LW', 'RCB|LCB'])

In [ ]:
# drop position_grouped column from the dataframes that will be used for the PCA analysis
for key, value in position_dfs.items():
    position_dfs[key] = value.drop('position_grouped')

In [53]:
# test 1 dataframe to see if the data cleaning was done correctly
position_dfs['CB']

P90 Count HSR,P90 Count HSR 1,P90 Count HSR 2,P90 Count HSR OTIP,P90 Count HSR OTIP 1,P90 Count HSR OTIP 2,P90 Count HSR TIP,P90 Count HSR TIP 1,P90 Count HSR TIP 2,P90 Count High Acceleration,P90 Count High Acceleration 1,P90 Count High Acceleration 2,P90 Count High Acceleration OTIP,P90 Count High Acceleration OTIP 1,P90 Count High Acceleration OTIP 2,P90 Count High Acceleration TIP,P90 Count High Acceleration TIP 1,P90 Count High Acceleration TIP 2,P90 Count High Deceleration,P90 Count High Deceleration 1,P90 Count High Deceleration 2,P90 Count High Deceleration OTIP,P90 Count High Deceleration OTIP 1,P90 Count High Deceleration OTIP 2,P90 Count High Deceleration TIP,P90 Count High Deceleration TIP 1,P90 Count High Deceleration TIP 2,P90 Count Medium Acceleration,P90 Count Medium Acceleration 1,P90 Count Medium Acceleration 2,P90 Count Medium Acceleration OTIP,P90 Count Medium Acceleration OTIP 1,P90 Count Medium Acceleration OTIP 2,P90 Count Medium Acceleration TIP,P90 Count Medium Acceleration TIP 1,P90 Count Medium Acceleration TIP 2,P90 Count Medium Deceleration,P90 Count Medium Deceleration 1,P90 Count Medium Deceleration 2,P90 Count Medium Deceleration OTIP,P90 Count Medium Deceleration OTIP 1,P90 Count Medium Deceleration OTIP 2,P90 Count Medium Deceleration TIP,P90 Count Medium Deceleration TIP 1,P90 Count Medium Deceleration TIP 2,P90 Count Sprint,P90 Count Sprint 1,P90 Count Sprint 2,P90 Count Sprint OTIP,P90 Count Sprint OTIP 1,P90 Count Sprint OTIP 2,P90 Count Sprint TIP,P90 Count Sprint TIP 1,P90 Count Sprint TIP 2,P90 Distance,P90 Distance 1,P90 Distance 2,P90 Distance OTIP,P90 Distance OTIP 1,P90 Distance OTIP 2,P90 Distance TIP,P90 Distance TIP 1,P90 Distance TIP 2,P90 HSR Distance,P90 HSR Distance 1,P90 HSR Distance 2,P90 HSR Distance OTIP,P90 HSR Distance OTIP 1,P90 HSR Distance OTIP 2,P90 HSR Distance TIP,P90 HSR Distance TIP 1,P90 HSR Distance TIP 2,P90 Minutes,P90 Minutes 1,P90 Minutes 2,P90 Minutes OTIP,P90 Minutes OTIP 1,P90 Minutes OTIP 2,P90 Minutes TIP,P90 Minutes TIP 1,P90 Minutes TIP 2,P90 Running Distance,P90 Running Distance 1,P90 Running Distance 2,P90 Running Distance OTIP,P90 Running Distance OTIP 1,P90 Running Distance OTIP 2,P90 Running Distance TIP,P90 Running Distance TIP 1,P90 Running Distance TIP 2,P90 Sprinting Distance,P90 Sprinting Distance 1,P90 Sprinting Distance 2,P90 Sprinting Distance OTIP,P90 Sprinting Distance OTIP 1,P90 Sprinting Distance OTIP 2,P90 Sprinting Distance TIP,P90 Sprinting Distance TIP 1,P90 Sprinting Distance TIP 2,P90 competition.id,P90 season.end_year,P90 season.id,P90 season.start_year,PSV-99,competition_region_division
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
31.439433,38.11137,25.149701,72.3182,71.377073,73.502722,36.898994,42.481457,30.0,7.397514,9.527842,5.389222,10.84773,12.97765,8.166969,6.708908,6.06878,7.5,12.02096,17.150116,7.185629,10.84773,12.97765,8.166969,6.708908,6.06878,7.5,110.962704,116.239678,105.988024,151.86822,155.731795,147.005445,107.342527,103.169252,112.5,86.920785,81.939445,91.616766,101.24548,97.332372,106.170599,83.861349,72.825354,97.5,7.397514,7.622274,7.185629,18.07955,19.466474,16.333938,6.708908,6.06878,7.5,8903.832323,9722.210459,8132.335329,11690.237043,12075.702956,11205.08167,10740.96161,11020.903574,10395.0,423.507654,472.580987,377.245509,950.984331,895.457823,1020.871143,409.243384,479.433581,322.5,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,1035.651906,1200.508152,880.239521,1974.286862,2011.535689,1927.404719,1288.110324,1420.094403,1125.0,178.465016,182.934575,174.251497,339.89554,441.240087,212.341198,181.140514,60.687795,330.0,14.795027,1869

In [57]:
X = position_dfs['CB'].drop("competition_region_division")
y = position_dfs['CB']["competition_region_division"]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

Scaling

In [59]:
#initialize the scaler
scaler=StandardScaler()

#use the scaler on the training features
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [ ]:
#starting the pca